# 1.加载数据
    pd.read_sql 优化问题 https://yq.aliyun.com/articles/607391
    
# 2.处理数据

In [223]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 1.1 从csv读取数据

In [224]:
def load_data(filename):
    print("\n正在加载数据......")
    data = pd.read_csv(filename)
    print("数据加载完成!!!\n")
    return data

data = load_data('ratings.csv')


正在加载数据......
数据加载完成!!!



## 1.2 划分测试集和训练集 

In [72]:

def split_data(data):
    print("\n正在划分数据集......")
    train_data, test_data = train_test_split(data, test_size=0.2)
    print("测试集{}条\t 训练集{}条".format(test_data.count()[0], train_data.count()[0]))
    print("划分数据集完成!!!\n")
    return train_data, test_data

train_data, test_data = split_data(data)


正在划分数据集......
测试集20168条	 训练集80668条
划分数据集完成!!!



## 2.1 构建movie_user 表 {}

In [84]:
train_data.head()

,userId,movieId,rating,timestamp
22044,144,1682,2.5,1137323966
44963,298,7361,2.5,1447518020
28012,192,110,3.0,835128955
59499,387,1562,1.5,1095041022
56516,375,1923,4.5,1225314346


In [173]:
pivot_movie_user = pd.pivot_table(train_data, index=["movieId","userId"], values="rating")
print('ratings 的 值 {} '.format(list(pivot_movie_user.values)[0][0]))
pivot_movie_user.query('movieId == 1')

ratings 的 值 4.0 


rating
movieId userId        
1       1          4.0
        7          4.5
        17         4.5
        18         3.5
        21         3.5
...                ...
        604        3.0
        605        4.0
        607        4.0
        608        2.5
        609        3.0

[175 rows x 1 columns]

In [129]:
pd.pivot_table(train_data, index=["movieId","userId"], values="rating").iloc[0:1]

,,rating
movieId,userId,
1,1,4.0


In [225]:
import pymysql
import time
con = pymysql.connect(host='127.0.0.1', user='root', password='wozhiai0',db='movies')

In [226]:
n1 = time.time()
Ratings = pd.read_sql("select * from rate", con)
n2 = time.time()

print(n2-n1)

14.706554174423218


In [227]:
n1 = time.time()
pivot_movie_user = pd.pivot_table(Ratings, index=["movie_id","user_id"], values="star")
n2 = time.time()
print(n2-n1)

0.5023059844970703


In [228]:
n1 = time.time()
pivot_user_movie = pd.pivot_table(Ratings, index="user_id",columns='movie_id', values="star")
n2 = time.time()
print(n2-n1)

22.574732065200806


In [229]:
ratings_movies = list(Ratings.movie_id.unique())

In [230]:
n1 = time.time()
Movies = pd.read_sql("select * from info", con)
n2 = time.time()

print(n2-n1)

3.2614080905914307


In [231]:
movies = list(Movies.id.unique())

In [222]:
print("ratings中的movieId 有{}条 而Info的movieId 有{}条 相差{}条".format(len(ratings_movies), len(Movies), len(ratings_movies)-len(Movies)))

r_m = list(set(ratings_movies).difference(set(movies)))
for i in r_m:
    print(i,end='')
    print()
print(len(r_m))
print('以上数据需要重新爬取')



ratings中的movieId 有50084条 而Info的movieId 有48801条 相差1283条
26935296
30285826
3080195
3162120
1916936
3612682
4309001
26935308
1302541
1875985
27123733
1957911
19963931
3162140
1302557
21340191
2277409
20561958
2080810
30433323
26378296
3555387
2170939
10428478
27099199
34930755
26656839
26738762
1302604
10428494
6430803
3711072
19955810
25837667
26353773
19955824
26247281
1876081
30220403
1982581
25731191
1441915
26673279
6013063
2990221
27000981
26312853
30458008
2998425
2326683
26271900
1441950
26575007
4882601
1302698
4882608
4882609
4882610
4882611
1441973
4882613
26386613
4735164
30351548
27140286
26706109
4882625
25977027
4882629
26353862
5193926
1786054
1433797
3670218
26935498
26353868
10461388
1302735
4882640
25698513
4751571
26681555
2990294
26599641
20152546
30163175
25870571
26779885
5161198
25805039
1302766
26886387
30179572
2130165
26657015
2302211
4833542
26894599
27107596
3801357
6537490
34439448
3088667
26870044
3735839
26935584
1401124
24838437
4833574
1401129
3408170
153

10827162
30463396
26195366
3765673
24876460
3765684
26785206
1390008
10606008
1308093
25834945
26662337
2160071
26662345
3159498
26875339
5092816
3012048
25843157
1439190
27186648
1439192
26858970
1299930
1299933
1439200
1971681
26588644
27596261
27203047
26678760
2299368
24745449
4847079
1308140
4847089
2299379
6428148
4847093
3012086
10483195
2373118
5969411
30414347
2160144
26326546
24753685
26326551
34960921
3806748
30479900
1308190
30414367
10458667
27194924
1422894
4896303
27620913
25933363
6755897
6755898
27063867
5355067
2086458
2356794
1963578
25949760
26383941
10745415
1291848
1291849
27121224
1308236
26842702
2299474
4740691
3765843
6829658
2299482
10442332
22861406
1291873
25728612
27194983
27194985
26433135
26367601
30463601
2053745
2274932
2365045
25753204
30135924
23074430
26826370
1898121
5355146
25851541
21370519
26392215
1316505
25982617
5985944
25663130
3446429
25851555
1578664
26842796
2274989
1791662
30463663
30365359
27104945
3684022
1857206
1300160
3503813
256631

# 数据库需要再爬取的电影信息条目 

In [232]:
import pymysql
import time
con = pymysql.connect(host='139.196.137.235', user='root', password='wozhiai0',db='movies')

In [233]:
Movies = pd.read_sql("select id from info", con)

In [234]:
ratings = pd.read_sql("select movie_id from rate", con)

In [239]:
movies = list(Movies.id.unique())

In [ ]:
ratings.movie_